# 02 | 病理图像处理（TITAN）

本 Notebook 讲解如何使用 **TITAN** 进行病理图像特征提取与分类。
我们会：

1. 组织病理图像 Patch 数据
2. 使用 TITAN 做特征提取
3. 训练一个轻量分类器
4. 评估准确率与混淆矩阵

> 注意：TITAN 模型较大，建议使用 GPU。


## 0. 环境准备

**做什么**：安装图像处理与 Hugging Face 依赖。
**为什么**：TITAN 可通过 Hugging Face 直接加载。
**结果**：安装后即可调用模型。

In [ ]:
!pip install torch torchvision scikit-learn pillow transformers


## 0.5 ✅ Notebook 冒烟测试（可选）

**做什么**：用最小代码验证“数据 -> 特征 -> 线性分类”的流程。
**为什么**：确认你当前环境可以跑通核心逻辑。
**结果**：输出一个可运行的准确率。

> 这一小节不依赖任何第三方库。

In [ ]:
# 两类简单“特征”（不依赖 NumPy）
tumor = [
    [1.0, 1.1, 0.9, 1.0],
    [1.2, 1.0, 1.1, 0.9],
]
normal = [
    [0.1, 0.0, 0.2, 0.1],
    [0.0, 0.1, 0.1, 0.2],
]

def mean(vec):
    return sum(vec) / len(vec)

X = tumor + normal
y = [1] * len(tumor) + [0] * len(normal)
pred = [1 if mean(v) > 0.5 else 0 for v in X]
acc = sum(p == t for p, t in zip(pred, y)) / len(y)
print('Smoke accuracy:', acc)


## 1. 数据准备

### 1.1 Patch 数据组织方式

**推荐目录**：
```
data/titan/
  train/
    tumor/
    normal/
  val/
    tumor/
    normal/
```

**要点**：
- 每张病理大图切成固定大小的 patch（如 256×256）
- 根据病理标注把 patch 放进对应类别文件夹

### 1.2 玩具数据（可运行示例）
我们生成两类简单的颜色块 patch 来跑通流程。

In [ ]:
import os
import numpy as np
from PIL import Image

np.random.seed(42)
os.makedirs('data/titan/train/tumor', exist_ok=True)
os.makedirs('data/titan/train/normal', exist_ok=True)
os.makedirs('data/titan/val/tumor', exist_ok=True)
os.makedirs('data/titan/val/normal', exist_ok=True)

def save_patch(path, color):
    img = np.ones((256, 256, 3), dtype=np.uint8) * np.array(color, dtype=np.uint8)
    Image.fromarray(img).save(path)

for i in range(10):
    save_patch(f'data/titan/train/tumor/{i}.png', [180, 50, 50])
    save_patch(f'data/titan/train/normal/{i}.png', [50, 180, 50])

for i in range(4):
    save_patch(f'data/titan/val/tumor/{i}.png', [180, 50, 50])
    save_patch(f'data/titan/val/normal/{i}.png', [50, 180, 50])

print('Toy pathology patches created.')


## 2. 加载 TITAN 模型

**做什么**：从 Hugging Face 下载 TITAN 预训练模型。
**为什么**：TITAN 是专门针对病理图像训练的强特征提取器。
**结果**：可以把图像变成高维特征向量。

> 如果你使用的是其他发布源，可替换 `model_id`。


In [ ]:
import torch
from transformers import AutoImageProcessor, AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_id = 'MahmoodLab/TITAN'
processor = AutoImageProcessor.from_pretrained(model_id)
titan = AutoModel.from_pretrained(model_id).to(device)
titan.eval()


## 3. 数据加载与特征提取

**做什么**：读取 patch，提取特征。
**为什么**：我们将用这些特征训练一个轻量分类器。
**结果**：得到 `X_train` 与 `X_val`。


In [ ]:
from glob import glob
from PIL import Image

def load_images(folder):
    paths = sorted(glob(os.path.join(folder, '*.png')))
    images = [Image.open(p).convert('RGB') for p in paths]
    return images

train_tumor = load_images('data/titan/train/tumor')
train_normal = load_images('data/titan/train/normal')
val_tumor = load_images('data/titan/val/tumor')
val_normal = load_images('data/titan/val/normal')

def extract_features(images):
    inputs = processor(images=images, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = titan(**inputs)
    feats = outputs.last_hidden_state.mean(dim=1)
    return feats.cpu()

X_train = torch.cat([extract_features(train_tumor), extract_features(train_normal)], dim=0)
y_train = torch.tensor([1] * len(train_tumor) + [0] * len(train_normal))

X_val = torch.cat([extract_features(val_tumor), extract_features(val_normal)], dim=0)
y_val = torch.tensor([1] * len(val_tumor) + [0] * len(val_normal))

print('Train features:', X_train.shape, 'Val features:', X_val.shape)


## 4. 训练轻量分类器

**做什么**：使用线性层进行分类。
**为什么**：TITAN 提供强特征，简单分类器即可。
**结果**：得到训练好的分类器参数。


In [ ]:
import torch.nn as nn
import torch.optim as optim

clf = nn.Linear(X_train.shape[1], 2).to(device)
optimizer = optim.Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

X_train_device = X_train.to(device)
y_train_device = y_train.to(device)

clf.train()
for epoch in range(5):
    logits = clf(X_train_device)
    loss = loss_fn(logits, y_train_device)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch}: loss={loss.item():.4f}')


## 5. 评估与混淆矩阵

**做什么**：计算准确率并输出混淆矩阵。
**为什么**：帮助你理解模型是否偏向某类。
**结果**：得到指标。


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

clf.eval()
with torch.no_grad():
    logits = clf(X_val.to(device))
    preds = torch.argmax(logits, dim=1).cpu().numpy()

acc = accuracy_score(y_val.numpy(), preds)
cm = confusion_matrix(y_val.numpy(), preds)
print('Validation Accuracy:', acc)
print('Confusion Matrix:', cm)
